In [ ]:
# Install required packages
!pip install transformers datasets torch box ensure pyyaml

In [ ]:
# Import necessary libraries
import os
import yaml
from box import ConfigBox
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

In [ ]:
# Utility functions
def read_yaml(path_to_yaml: Path) -> ConfigBox:
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            return ConfigBox(content)
    except Exception as e:
        raise e

def create_directories(path_to_directories: list, verbose=True):
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)

In [ ]:
# Configuration and parameters
config = read_yaml('config.yaml')
params = read_yaml('params.yaml')

In [ ]:
# Model Trainer Class
class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        # Verify GPU availability
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f'Using device: {device}')

        # Initialize model and tokenizer
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Load dataset
        dataset = load_from_disk(self.config.data_path)

        # Training setup
        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=params.TrainingArguments.num_train_epochs,
            per_device_train_batch_size=params.TrainingArguments.per_device_train_batch_size,
            evaluation_strategy=params.TrainingArguments.evaluation_strategy,
            eval_steps=params.TrainingArguments.eval_steps,
            save_steps=params.TrainingArguments.save_steps,
            gradient_accumulation_steps=params.TrainingArguments.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            train_dataset=dataset['test'],
            eval_dataset=dataset['validation']
        )

        # Start training
        trainer.train()

        # Save model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'tokenizer'))

In [ ]:
# Execute training
trainer = ModelTrainer(config.model_trainer)
trainer.train()